In [ ]:
# Install Open3D 
!pip install open3d -q


In [ ]:
import numpy as np
import open3d as o3d

def create_voxel_scene(size=(32, 32, 16)):
    voxels = np.zeros(size)

    # Add a ground plane at z = 0
    voxels[:, :, 0] = 1

    # Add a wall
    voxels[10:20, 16, 1:5] = 1

    # Add a vertical tower
    voxels[5, 5, 1:10] = 1

    return voxels


In [ ]:
def voxels_to_mesh(voxel_grid):
    mesh = o3d.geometry.TriangleMesh()
    for x in range(voxel_grid.shape[0]):
        for y in range(voxel_grid.shape[1]):
            for z in range(voxel_grid.shape[2]):
                if voxel_grid[x, y, z] == 1:
                    cube = o3d.geometry.TriangleMesh.create_box(width=1, height=1, depth=1)
                    cube.translate((x, y, z))
                    cube.compute_vertex_normals()
                    mesh += cube
    return mesh


In [ ]:
def render_scene(mesh, camera_pos=[60, 40, 30], lookat=[16, 16, 4], width=512, height=512):
    # Set up scene
    scene = o3d.visualization.rendering.Open3DScene(o3d.visualization.rendering.Renderer())
    mat = o3d.visualization.rendering.MaterialRecord()
    mat.shader = "defaultLit"
    
    mesh.compute_vertex_normals()
    scene.scene.add_geometry("scene", mesh, mat)
    
    # Camera
    center = o3d.geometry.AxisAlignedBoundingBox.create_from_triangle_mesh(mesh).get_center()
    eye = np.array(camera_pos)
    up = np.array([0, 0, 1])

    scene.scene.camera.look_at(lookat, eye, up)
    
    # Offscreen render
    img = scene.scene.render_to_image(width, height)
    return np.asarray(img)


In [ ]:
voxels = create_voxel_scene()
mesh = voxels_to_mesh(voxels)
image = render_scene(mesh)

import matplotlib.pyplot as plt
plt.imshow(image)
plt.title("Rendered Voxel Scene")
plt.axis('off')
plt.show()


In [ ]:
# Placeholder for depth map (real Z-buffer rendering needs custom pipeline)
depth_map = np.linalg.norm(np.indices(voxels.shape).T - np.array([16,16,0]), axis=-1)
np.save("/content/depth_0.npy", depth_map)
